In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

function printer(x::Any, verbose="")
    if verbose != ""
        println(verbose)
    end
    show(stdout, "text/plain", x)
    println("")
end

using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
gr();

In [ ]:
motion_noise_stds = Dict("vv" => 0.001, "vω" => 0.001, "ωv" => 0.13, "ωω" => 0.001)
function ch05_mcl06(motion_noise_stds)
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    world = World(xlim, ylim)
    # robot side
    initial_pose = [-2.0, 0.0, 0.0]
    estimator = Mcl(initial_pose, 100, motion_noise_stds)
    circling = EstimatorAgent(0.1, 0.0, dt, estimator)
    robot = RealRobot(initial_pose, circling, nothing; color="red")
    push!(world, robot)
    anim = @animate for i in 1:400
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        p = draw(world, annota)
        z = observations(robot.sensor_, robot.pose_)
        v, ω = decision(circling, z)
        state_transition(robot, v, ω, dt; move_noise=true, vel_bias_noise=true)
    end
    gif(anim, "images/ch05_mcl06.gif", fps=10)
end
ch05_mcl06(motion_noise_stds)